# Plant Leaf Disease Detection
Mayur Kr. Garg

## Imports

In [1]:
import os
import tensorflow as tf

In [2]:
tf.config.experimental.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Constants

In [3]:
BASE_DIR = "../../Datasets/New Plant Diseases (Augmented)"
TRAIN_DIR = os.path.join(BASE_DIR, "train")
VAL_DIR = os.path.join(BASE_DIR, "valid")
TEST_DIR = os.path.join(BASE_DIR, "test")

## Data Loading

### Data Distribution

#### Train

In [4]:
total_train = 0

for d in os.listdir(TRAIN_DIR):
    path = os.path.join(TRAIN_DIR, d)
    img_count = len(os.listdir(path))
    total_train += img_count
    print(f'{d:50} : {img_count}')

print(f'\nTotal training images: {total_train}')

Apple___Apple_scab                                 : 2016
Apple___Black_rot                                  : 1987
Apple___Cedar_apple_rust                           : 1760
Apple___healthy                                    : 2008
Blueberry___healthy                                : 1816
Cherry_(including_sour)___healthy                  : 1826
Cherry_(including_sour)___Powdery_mildew           : 1683
Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot : 1642
Corn_(maize)___Common_rust_                        : 1907
Corn_(maize)___healthy                             : 1859
Corn_(maize)___Northern_Leaf_Blight                : 1908
Grape___Black_rot                                  : 1888
Grape___Esca_(Black_Measles)                       : 1920
Grape___healthy                                    : 1692
Grape___Leaf_blight_(Isariopsis_Leaf_Spot)         : 1722
Orange___Haunglongbing_(Citrus_greening)           : 2010
Peach___Bacterial_spot                             : 1838
Peach___health

#### Validation

In [5]:
total_val = 0

for d in os.listdir(VAL_DIR):
    path = os.path.join(VAL_DIR, d)
    img_count = len(os.listdir(path))
    total_val += img_count
    print(f'{d:50} : {img_count}')

print(f'\nTotal validation images: {total_val}')

Apple___Apple_scab                                 : 504
Apple___Black_rot                                  : 497
Apple___Cedar_apple_rust                           : 440
Apple___healthy                                    : 502
Blueberry___healthy                                : 454
Cherry_(including_sour)___healthy                  : 456
Cherry_(including_sour)___Powdery_mildew           : 421
Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot : 410
Corn_(maize)___Common_rust_                        : 477
Corn_(maize)___healthy                             : 465
Corn_(maize)___Northern_Leaf_Blight                : 477
Grape___Black_rot                                  : 472
Grape___Esca_(Black_Measles)                       : 480
Grape___healthy                                    : 423
Grape___Leaf_blight_(Isariopsis_Leaf_Spot)         : 430
Orange___Haunglongbing_(Citrus_greening)           : 503
Peach___Bacterial_spot                             : 459
Peach___healthy                

#### Test

In [6]:
total_test = len(os.listdir(TEST_DIR))

for i in os.listdir(TEST_DIR):
    print(i)

print(f'\nTotal testing images: {total_test}')

AppleCedarRust1.JPG
AppleCedarRust2.JPG
AppleCedarRust3.JPG
AppleCedarRust4.JPG
AppleScab1.JPG
AppleScab2.JPG
AppleScab3.JPG
CornCommonRust1.JPG
CornCommonRust2.JPG
CornCommonRust3.JPG
PotatoEarlyBlight1.JPG
PotatoEarlyBlight2.JPG
PotatoEarlyBlight3.JPG
PotatoEarlyBlight4.JPG
PotatoEarlyBlight5.JPG
PotatoHealthy1.JPG
PotatoHealthy2.JPG
TomatoEarlyBlight1.JPG
TomatoEarlyBlight2.JPG
TomatoEarlyBlight3.JPG
TomatoEarlyBlight4.JPG
TomatoEarlyBlight5.JPG
TomatoEarlyBlight6.JPG
TomatoHealthy1.JPG
TomatoHealthy2.JPG
TomatoHealthy3.JPG
TomatoHealthy4.JPG
TomatoYellowCurlVirus1.JPG
TomatoYellowCurlVirus2.JPG
TomatoYellowCurlVirus3.JPG
TomatoYellowCurlVirus4.JPG
TomatoYellowCurlVirus5.JPG
TomatoYellowCurlVirus6.JPG

Total testing images: 33
